# Deezer Music Recommender System

In [34]:
# Basic libraries
import gzip
import tarfile
import pickle
import operator
import warnings
import pandas as pd
import numpy as np
import time
from collections import defaultdict
import matplotlib.pyplot as plt

# Classifier Libraries
from sklearn.svm import SVC

# Evaluation Metrics
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
# Disable warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
TRAIN_DATA_PATH = "/home/lmelvix/projects/deezer-music-recommendation/data/train.csv"
TEST_DATA_PATH = "/home/lmelvix/projects/deezer-music-recommendation/data/test.csv"
EXTRA_INFO_PATH = "/home/lmelvix/projects/deezer-music-recommendation/data/extra_infos.json"

# Read Train and Test dataset into DataFrame

In [4]:
train_df = pd.read_csv(TRAIN_DATA_PATH)
test_df = pd.read_csv(TEST_DATA_PATH)

# Preprocess JSON file and read into DataFrame
with open(EXTRA_INFO_PATH, 'rb') as f:
    data = f.readlines()
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"
extra_df = pd.read_json(data_json_str)

In [5]:
train_df.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened
0,25471,1480597215,222606,41774,12,20040704,1,0,223,0,0,9241,55164,29,0
1,25571,1480544735,250467,43941,0,20060301,2,1,171,0,0,16547,55830,30,1
2,16,1479563953,305197,48078,1,20140714,2,1,149,1,1,7665,2704,29,1
3,7,1480152098,900502,71521,0,20001030,0,0,240,0,1,1580,938,30,0
4,7,1478368974,542335,71718,0,20080215,0,0,150,0,1,1812,2939,24,1


# Filter Dataset

In [14]:
# TODO: Analyze and filter dataset

# Feature Extraction

In [24]:
# TODO: Identify new features

# Build Feature Vector

In [6]:
predictors = ["genre_id"]

# Split Train and Validation 

In [10]:
# Shuffle and split Training dataset into 80-20
train_percent = 0.01
valid_percent = 0.01
perm = np.random.permutation(train_df.index)
m = len(train_df)
train_end = int(train_percent * m)
valid_end = int(valid_percent * m) + train_end
mdata_train = train_df.ix[perm[:train_end]]
mdata_valid = train_df.ix[perm[train_end:valid_end]]

# Build Model

In [33]:
classifier = SVC()
svc_status = classifier.fit(mdata_train[predictors], mdata_train['is_listened'])

In [11]:
predict = classifier.predict(mdata_valid[predictors])

# Evaluate Model for Validation Set

In [13]:
accuracy = accuracy_score(mdata_valid['is_listened'], predict)
print accuracy

0.682806794729


# Predict for Test Data

In [14]:
#TODO: Train classifier with complete dataset
#TODO: Prepare test data correspondingly

In [15]:
test_predict = classifier.predict(test_df[predictors])

# Prepare Submission File 

In [26]:
submission_file_name = "deezer_predict_" + time.strftime("%Y%m%d_%H%M%S") + ".csv"
test_df['is_listened'] = test_predict

In [32]:
to_push = ["sample_id", "is_listened"]
test_df[to_push].to_csv(submission_file_name, index=False)